This notebook trains a surrogate model using TensorFlow.  The resulting model is then saved and converted to a TensorFlowJS format.  The model is then loaded and used to make predictions on new data on a webpage.

To run this notebooks, use the `ws-env` virtual environment, which can be built using the `environment.yml` file located in the same directory as this notebook.

In [1]:
# imports
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split


In [2]:
# Read in training data
df = pd.read_csv('ExampleData.csv')
# Split the X and y variables
X = df[["r", "age", "start_year", "num_years", "mortality", "fertility", "productivity"]].values
y = df[["NPV_100", "GDP_20"]].values

In [3]:
# train-test split for model evaluation
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, shuffle=True
)

In [4]:
# Layer setting
num_input = X.shape[1]
num_hidden1 = 10 * num_input
num_hidden2 = 10 * num_input
num_output = y.shape[1]
layers_dim = [num_input, num_hidden1, num_hidden2, num_output]
print("Dimensions of each layer are {}".format(layers_dim))

Dimensions of each layer are [7, 70, 70, 2]


In [5]:
# Define the neural network
# We use [Keras](https://www.tensorflow.org/guide/keras) to define the
# neural network
nn = tf.keras.Sequential(
    [
        keras.layers.Dense(
            num_hidden1, activation="relu", input_shape=(num_input,)
        ),
        keras.layers.Dense(num_hidden2, activation="relu"),
        keras.layers.Dense(num_output),
    ]
)
print(nn.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 70)                560       
                                                                 
 dense_1 (Dense)             (None, 70)                4970      
                                                                 
 dense_2 (Dense)             (None, 2)                 142       
                                                                 
Total params: 5672 (22.16 KB)
Trainable params: 5672 (22.16 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [6]:
loss_fn = tf.keras.losses.MeanSquaredError()
nn.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [7]:
nn.fit(X_train, y_train, epochs=200)

Epoch 1/200
354/354 [==============================] - 0s 597us/step - loss: 5.3187 - accuracy: 0.6718
Epoch 2/200
354/354 [==============================] - 0s 566us/step - loss: 0.3811 - accuracy: 0.7154
Epoch 3/200
354/354 [==============================] - 0s 733us/step - loss: 0.1865 - accuracy: 0.7197
Epoch 4/200
354/354 [==============================] - 0s 965us/step - loss: 0.0924 - accuracy: 0.7309
Epoch 5/200
354/354 [==============================] - 0s 648us/step - loss: 0.0429 - accuracy: 0.7473
Epoch 6/200
354/354 [==============================] - 0s 792us/step - loss: 0.0213 - accuracy: 0.7538
Epoch 7/200
354/354 [==============================] - 0s 764us/step - loss: 0.0117 - accuracy: 0.7597
Epoch 8/200
354/354 [==============================] - 0s 646us/step - loss: 0.0074 - accuracy: 0.7611
Epoch 9/200
354/354 [==============================] - 0s 633us/step - loss: 0.0055 - accuracy: 0.7613
Epoch 10/200
354/354 [==============================] - 0s 851us/step - l

In [8]:
nn.evaluate(X_test,  y_test, verbose=2)

152/152 - 0s - loss: 6.3458e-04 - accuracy: 0.4886 - 219ms/epoch - 1ms/step


[0.0006345755537040532, 0.48855432868003845]

In [9]:
# save full model - not just weights
tf.keras.Model.save(nn, "SL_model_full.h5", save_format="h5")

/Users/jason.debacker/anaconda3/envs/dynviz-dev/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
!tensorflowjs_converter --input_format=keras SL_model_full.h5 ./tf_model/

In [11]:
# Read model back in
model = keras.models.load_model("SL_model_full.h5")

In [12]:
# use model to make predictions
predictions = model.predict(X_test)
print(predictions)

152/152 [==============================] - 0s 469us/step
[[ 3.6908774  3.7228234]
 [ 5.058204   5.0879483]
 [ 6.3558087  6.4215407]
 ...
 [ 7.7550025  7.786715 ]
 [ 2.8876476  5.7926497]
 [ 6.659769  13.341977 ]]


In [13]:
# use model to make a single prediction
x = np.array([[2.89386640e-02, 4.00000000e+01, 1.00000000e+01, 2.00000000e+01,
       2.76861914e-01, 5.76141576e+00, 6.13201966e+00]])
print(model.predict(x)[0]) # Print the first (only) result

1/1 [==============================] - 0s 12ms/step
[4.7694187 4.803929 ]
